In [22]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [3]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv("cities.csv")

# Clean up dataframe 
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df.dropna()
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Kousseri,72,CM,1580932050,14,12.08,15.03,75.20,5.82
1,Kathmandu,20,NP,1580931800,87,27.72,85.32,44.60,3.36
2,San Ramon,40,US,1580932025,50,37.78,-121.98,61.00,3.78
3,Laas,68,IT,1580932054,61,46.62,10.70,46.40,8.05
4,Qaanaaq,0,GL,1580932055,43,77.48,-69.36,-19.95,4.18


### Humidity Heatmap

In [5]:
# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["Lat", "Lng"]]

# Convert humidity to float
humidity = cities_df["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100, point_radius = 4)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Create a dataframe of cities that meet my criteria
retire_df = cities_df.loc[(cities_df["Humidity"] >= 50)&(cities_df["Humidity"] <= 81) &
                          (cities_df["Max Temp"]>=60)&(cities_df["Max Temp"]<=80) &
                          (cities_df["Wind Speed"] <= 40), :].reset_index()

# Remove index column that appears with reset_index()
retire_df = retire_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]

# Preview dataframe
retire_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,San Ramon,40,US,1580932025,50,37.78,-121.98,61.00,3.78
1,Hilo,90,US,1580932067,78,19.73,-155.09,75.00,12.75
2,Port Blair,100,IN,1580932068,67,11.67,92.75,78.69,9.86
3,Kruisfontein,74,ZA,1580932072,80,-34.00,24.73,65.30,6.08
4,Hong Kong,40,HK,1580932082,72,22.29,114.16,63.00,9.17
...,...,...,...,...,...,...,...,...,...
63,Opuwo,98,NaN,1580932766,63,-18.06,13.84,71.71,6.42
64,Soe,59,ID,1580932819,75,-9.86,124.28,74.17,3.60
65,Kanniyakumari,98,IN,1580932823,76,8.08,77.57,79.32,19.10
66,Bull Savanna,14,JM,1580932843,66,17.89,-77.59,78.03,15.46


In [ ]:
Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [21]:
hotel_df = retire_df.dropna()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,San Ramon,40,US,1580932025,50,37.78,-121.98,61.00,3.78
1,Hilo,90,US,1580932067,78,19.73,-155.09,75.00,12.75
2,Port Blair,100,IN,1580932068,67,11.67,92.75,78.69,9.86
3,Kruisfontein,74,ZA,1580932072,80,-34.00,24.73,65.30,6.08
4,Hong Kong,40,HK,1580932082,72,22.29,114.16,63.00,9.17
...,...,...,...,...,...,...,...,...,...
62,Peniche,67,PT,1580932761,76,39.36,-9.38,64.00,9.73
64,Soe,59,ID,1580932819,75,-9.86,124.28,74.17,3.60
65,Kanniyakumari,98,IN,1580932823,76,8.08,77.57,79.32,19.10
66,Bull Savanna,14,JM,1580932843,66,17.89,-77.59,78.03,15.46


In [23]:
# Create dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the latitude and longitude to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration and leave original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # Convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Use try-except to skip any that are missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAK9LNQm7xO0ERWXKWvBY1vXiQRWDsekWcJZC8MFRo8ZdECNSwQ8hYgEsou5q-_5_zVzt1qYuEd-A-1nQnN8KoJsHBPY1ZLEQFur4m4spuYUm0pPx3TK56r5dLsy9RPy_wad13G0u3v0_VkAPu_sM9MdzRJu_VsCgkIFIwT4V-BcVGIohgYdriCqxt8vsbzELZOelWrs2ww8IZ_vR90UDwmTxvmorAl2NsdwlIgsLMggu0J1zuzwbvCD0piBLRGIuQYdW9R71BkKl98Ed9tRVLAPQ6PllE8QVLwB7-RQoOcDbD_MyOa6M6F8ZCkDoMMqu_PQ5LcPHaNWPq0D8VQhVPeGsrKYFlQMyoSy7uyDvfFnopFgv4GKooAE3qSK-MqMdN0bt10GxDlrEFa5B3u_brIISELM0-s2abd_lUQlZyir3sjkaFCmHG97h65mG8wZyhB1-1PqNhQ5U",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 37.700278,
                    "lng": -121.918159
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.70180842989272,
                        "lng": -121.9169862701073
                    },
                    "southwest": {
                        "lat": 37.69910877010728,
                      

C:\Users\mtqmc\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mtqmc\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAIvQaR6r6IzwMOVQanOjAGNw7EWD6J8-x9zUnUQO1QOQgsEAu5oJvzRnnvJ9DTy4-OSL9PYJ1xBcZtJUUKnUVCMb1PqTaTlVIT_kHR1U4olQnhZLGsfQx8Wz4K1QPBo2jOE38G2bXoQUG-7wvBgWAgkVOUHIG5lW626Zb79stD_wljiA8WXpRxhBD-U6Qf30sYoVnS9xEJlFuxl3l7a6CFZqdrpOJ2tRPMYRmKwx49uEaXh4smLGrvOhk4ommpyHwDeDiDksfFdah663_I3e3j4Mv3-okT80gofoTjHJjAczKbJk9WEKlzHoSmNe20nTa7UMCNGkvYCdnYtAbHBnlQ0KBGI7ja5hevfFGz1L70RPGHRgTBVDGbLD_MrhMFQI8sDOo4HHKRrPXfDq_t0fBYoSEDlXLivMdzrETM10wrmyCfsaFKTcXcHqSUnyzaTvGew-miBxDyos",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 19.7295945,
                    "lng": -155.064893
                },
                "viewport": {
                    "northeast": {
                        "lat": 19.73059247989272,
                        "lng": -155.0635793701072
                    },
                    "southwest": {
                        "lat": 19.72789282010728,
                     

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAABbMCgjHh4oRpQf-rYLIYN-YkUGsTKzZv4Dk2KMGUQs28guBPrQDvPQspn0MeF8tyx9pj0ZlvEyuGRQuFquqN-EcDIC-Op7IsmZ_YlSM3pmaXZ5O7fS0gURx45VtVTKe0hAzit3dcdufzDCU4VHMPlDY0n5TRIqS9GrA0rmvEmeA_hCA3HbRQHejal5URuWq1Yf3zuDdjwkOq5theUyWr13wljdSED8R4hMXrYu-PhMAy9d9-1FwZ8Z9d_Gh0U66ZadoE_zLZ_gTC9ERwxz0d2t3vO69abYrT9ZRx4mI25crQXUDac4flmBpP7YJ18VlnKFVbW6U0iFHmYVunKNAq0bD773QDJ4O1d4RnXxlSCaig6iL0R_d7A6zgp2heOYVTqjrQ3DxtsRHAc0YT31QWQoSEEHSWThe4ONPio8YLRgnAnAaFP2koStnLq2xXbkyyF5jDoB8GLmy",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 11.983846,
                    "lng": 92.9559725
                },
                "viewport": {
                    "northeast": {
                        "lat": 11.98519582989272,
                        "lng": 92.95732232989272
                    },
                    "southwest": {
                        "lat": 11.98249617010728,
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAOObxqCOjd3tlehfwzD6LghWOsqqFzTu4Bs1odVvXLKk4BoYZQeyHT5nQ1sjzT2DaLyMqxNI40fzhGCF77sd0BB8DYo-l9AWjCqX5ub0PxkXxe0Kh9aY3ewrFU_SelBuZZTLPL8f5C2heeA4T58vA8efaIMa8BVjuIuqICts2QneOgkAxS2gv81tYl0BlOAPFFyzOgDaG2KIYkGsMmGRO6WOMlRrMGaRu1gTukAivcCpWXMn8InzPHLa-ifN03J8_xmW0b2cCEiIfZzXtYUIUXM3SGXyA3ZhUajLTWCPkYQgK7gxTvHoaDWYEXrhIAQQUOhFTPwNyIzfGf2UvQaBnd6WxW-O5EulQFUp5Su8hwHa0UjAhytFTQafRzREtMTgVf0DZnLNH3xKJ8PImXmkzi0SEIA95mJZKM9wv4rqMNQdqeYaFL4x9woU-whszJzqTCA-X5JsWWaG",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.031552,
                    "lng": 24.93181
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.03010412010728,
                        "lng": 24.93309422989272
                    },
                    "southwest": {
                        "lat": -34.03280377989272,
                       

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -9.8584619,
                    "lng": 124.2856955
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.856997820107276,
                        "lng": 124.2870118798927
                    },
                    "southwest": {
                        "lat": -9.85969747989272,
                        "lng": 124.2843122201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "afce16539a7c1dfba04cf4be45f07cea68fce4a2",
            "name": "Dena Hotel",
            "photos": [
                {
                    "height": 1536,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/106912138689800638266\">Anand Deo</

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAF8taUhXgz18SzHBWwmYw223nNlKQ4NY9InYJp4WyPCyxslk3yooXOUU9zOGZ7ikTfh-uD4o5AmWNjcHSuWDeFdckkDoiNGQGn9zfqlvpoeUiDih8mQSg0Zo2k6aV0JwDImPjluquhwtsgBcZT3oosNxnocD_6tTv5-6ZVEMDsZOWAUv93K60sVD3zcPlGx2o6WUMIf8HN_H0Z6u4vD7X4R2ezI8NYDsHMna8fURn2akUZMXR7CfMJjnvDxKC7Fpd0T75mJTxasPmDTVeMwrjYugBxbFXsgojhnnMXjbhOfLvji7DCyFSNG1N0JwdFiddLemGctFYSzH6doOO1Fu-K9s8EpkKn1hqKBZcl2epGSgoIOontGYiCR0ANnxdaeILuIAP6oYLRH8MHt5DtzuiisSEBH97Wi2e115srxF_iaDc0IaFJXh6AZMKCXYf45-teHGltQcgKW5",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 8.432692,
                    "lng": 77.242453
                },
                "viewport": {
                    "northeast": {
                        "lat": 8.433947129892722,
                        "lng": 77.24335952989271
                    },
                    "southwest": {
                        "lat": 8.431247470107278,
                        "l

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAJD4CZBRadEWADamfaQlNwb8lUZnsTPLyZKuGYn83LWbmZ9nPtNokm5x6n3-RKg-gbOp0EKWq6-8XOMyqZY4LMLduMP1YtjnTbfuaPuV4_3hwLLK3o0g8RYw1DeOo0QdV8Z7E8D5S1KNRL1eKsiJ9-mtEZLWHvroHLEEHsgF6HOrwvjZSdn_Dk3u4lJF9NsFvdDYgLruFcH1xbQ_7avLPmO-4nY6UhCalpcTbcMN0Lik1B7rkD5kUX3zhMLWb2XlJtmA30rOeSv2E0g4jC5qBhxn-eW6a93FFms6tOxxFA61gesURlNe1MpOK6OuNfus-rZw4ADLF0L3mnNtE2cMdwaL21sKY7tK5aaGpoONZQtTyNwoFpTIlaxkCutQvFJzIShBeXl0Jdd9fjMyZXOsVAUSEBJepoHlig4X5YfCcZLQjMkaFL1g7GqzJk8rAv6GdY85pMfc6Cka",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 18.176108,
                    "lng": -78.03295399999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 18.17750952989272,
                        "lng": -78.03153847010728
                    },
                    "southwest": {
                        "lat": 18.17480987010728,
               

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAFRQk7uddjIuYFenFxNrnkR9xpXTN9kcaTiErVa9qqe1670BhfjZJTaLvIdeuGt7EETu7HiBZz2VZrNU_S1Fx6V1TOdZFfG8pOpAchxYPdFUTKdbylSyuPFp35wyuwnItw_H9AinJfAO6iqSzAuimp1Os-ZxljZhni64-z3oRp6BefMS8UiwAZcWc4ndWwThNu2d-i7jMhZS_nm54JS0efPLBtmu4pP9gBaQVNpHVob2LmPiWep6dSVMzrIFLdiZ0A_fC3zQ8pOZsIukDy5-8t0kgh18bqOpch0XZrm07qfHuD_Qi8q0yIadiL0ecM194J5uFIYROYzqVMILrM7NPwvX4u3zhjMYZKvRm1wBW05zkBKtT8qVfAw6krLIxsMioodvW9m9P7z3KflBUdh_fwUSEInUEXXD5PR17q1ekNgfuw8aFDkthrsq0FbOdPuvyt225tNzBjly",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.5537861,
                    "lng": 121.0246261
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.55504127989272,
                        "lng": 121.0256211298928
                    },
                    "southwest": {
                        "lat": 14.55234162010728,
                      

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,San Ramon,40,US,1580932025,50,37.78,-121.98,61.00,3.78,DoubleTree by Hilton Hotel Pleasanton at the Club
1,Hilo,90,US,1580932067,78,19.73,-155.09,75.00,12.75,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
2,Port Blair,100,IN,1580932068,67,11.67,92.75,78.69,9.86,"Taj Exotica Resort & Spa, Andamans"
3,Kruisfontein,74,ZA,1580932072,80,-34.00,24.73,65.30,6.08,Supertubes Bed & Breakfast Guest House Jeffrey...
4,Hong Kong,40,HK,1580932082,72,22.29,114.16,63.00,9.17,"The Langham, Hong Kong"
...,...,...,...,...,...,...,...,...,...,...
62,Peniche,67,PT,1580932761,76,39.36,-9.38,64.00,9.73,Bukubaki Eco Surf Resort
64,Soe,59,ID,1580932819,75,-9.86,124.28,74.17,3.60,Dena Hotel
65,Kanniyakumari,98,IN,1580932823,76,8.08,77.57,79.32,19.10,Anantya Resorts & Hotels Pvt Ltd
66,Bull Savanna,14,JM,1580932843,66,17.89,-77.59,78.03,15.46,Bluefields Bay Villas


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))